In [1]:
from ner.Datasets.utils import *
from ner.Datasets.Conll2003Dataset import *

%load_ext autoreload
%autoreload 2

[nltk_data] Downloading package punkt to /home/ketl2/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ketl2/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/ketl2/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# data_tr = Conll2003Dataset(utils, split = 'train')
# data_te = Conll2003Dataset(utils, split = 'test')

In [3]:
clean_data_te = Conll2003Dataset(split = 'test', cleaned = True)
clean_data_tr = Conll2003Dataset(split = 'train', cleaned = True)

Map:   0%|          | 0/1588 [00:00<?, ? examples/s]

Map:   0%|          | 0/1588 [00:00<?, ? examples/s]

Map:   0%|          | 0/1588 [00:00<?, ? examples/s]

Map:   0%|          | 0/1588 [00:00<?, ? examples/s]

Map:   0%|          | 0/1588 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/7577 [00:00<?, ? examples/s]

Map:   0%|          | 0/7577 [00:00<?, ? examples/s]

Map:   0%|          | 0/7577 [00:00<?, ? examples/s]

Map:   0%|          | 0/7577 [00:00<?, ? examples/s]

Map:   0%|          | 0/7577 [00:00<?, ? examples/s]

In [4]:
clean_data_te

In [5]:
save_conll_dataset(clean_data_te)
save_conll_dataset(clean_data_tr)

In [6]:
dataset_test = load_conll_dataset(cleaned= True)

In [6]:
dataset_train.dataset

Dataset({
    features: ['id', 'tokens', 'ner_tags', 'text', 'spans', 'llama_tokenized', 'llama_text', 'llama_text_2', 'sentence_embedding', 'entity_embeddings'],
    num_rows: 7577
})

## Testing few-shots

In [4]:
# dataset_train, dataset_llama_50 = load_conll_dataset(split = 'test')
# dataset_test = load_conll_dataset(split = 'test', cleaned=True)
dataset_test_train, dataset_test_test = dataset_test.train_test_split(test_size=20)

Map:   0%|          | 0/1568 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [5]:
from ner.Llama2_NER import Llama2_NER_FewShots_Entity
ner_ent = Llama2_NER_FewShots_Entity(None, utils, dataset_test_train, nb_few_shots= 16, with_verify=False, prompt_type="discussion")

In [6]:
# dataset_llama_50.select([8,10,22,24,25,44,49])
for i, sample in enumerate(dataset_test_test) :
    print(sample['text'])
    print(sample['spans'])
    print(ner_ent.get_prompts(sample['text'], [], 5))
    print()
    print("----------------------------------------------------")
    print()

Livshits 's words are an attempt to put pressure on the company . "
[['Livshits', 'PER']]


2023-11-02 13:24:16.946245: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-02 13:24:17.817504: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### SYSTEM : The task is to extract all the named entites in the follnwing sentence.
### USER : Your goal is to extract all the enities that are either person, organization, location or miscallaneous.
### ASSISTANT : Yes I can do that. Can you provide me examples ?  
### USER : Yes of course, there are some examples : 
 ### INPUT : <start_input> * The three parties in the government coalition have committed themselves to a real reform of Romania 's economy , Constantinescu said after the ceremony . <end_input>
### OUTPUT : <start_output> [['Romania', 'LOC'], ['Constantinescu', 'PER']] <end_output>

### INPUT : <start_input> He took over there , and took control of the red button controlling nuclear arms , in December 1991 when Mikhail Gorbachev resigned , marking the end of the Soviet Union . <end_input>
### OUTPUT : <start_output> [['Mikhail Gorbachev', 'PER'], ['Soviet Union', 'LOC']] <end_output>

### INPUT : <start_input> The street protests were the biggest seen in the capital sin